In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow-addons
!pip install pyyaml h5py
!pip install toolz scikit-allel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.2 MB 5.1 MB/s 


In [ ]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Tensorflow version 2.9.2


## Setup

In [ ]:
import os
# os.environ["MODIN_CPUS"] = "8"
# from distributed import Client
# client = Client()
import numpy as np
import math
import re
import itertools
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_addons as tfa
from sklearn import metrics
from sklearn.model_selection import KFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.applications import efficientnet as efn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from tensorflow.keras.constraints import Constraint
# import allel
from scipy.spatial.distance import squareform
%matplotlib inline   
from toolz import interleave
from tqdm import tqdm
import allel
from scipy.spatial.distance import squareform
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.model_selection import KFold,StratifiedKFold

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.9.2


## Hardware Config

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', TPU.master())
except ValueError:
    print('Running on GPU')
    TPU = None

if TPU:
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.experimental.TPUStrategy(TPU)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

N_REPLICAS = strategy.num_replicas_in_sync
# Number of computing cores, is 8 for a TPU V3-8
print(f'N_REPLICAS: {N_REPLICAS}')

Running on GPU
N_REPLICAS: 1


## Prepare the data

In [ ]:
root_dir = '[path]/'
new_data_header = ""
# get header
with open(root_dir + "ALL.chr22.mergedSV.v8.20130502.svs.genotypes.vcf", 'r') as f_in:
    # skip info
    for line_num in range(70):
        f_in.readline()

    new_data_header = f_in.readline()
# load data

# load genotype
genotypes = pd.read_csv(root_dir + "ALL.chr22.mergedSV.v8.20130502.svs.genotypes.vcf", comment='#', sep='\t', names=new_data_header.strip().split('\t'), header=None).iloc[:, 9:]
genotypes = genotypes.T
headers = genotypes.columns[:]
genotypes

,0,1,2,3,4,5,6,7,8,9,...,838,839,840,841,842,843,844,845,846,847
HG00096,3|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
HG00097,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
HG00099,0|0,0|0,0|0,0|0,0|0,0|1,0|1,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
HG00100,0|0,0|0,0|0,0|0,0|0,0|0,0|1,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
HG00101,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NA21137,3|0,0|0,0|0,0|0,0|0,0|0,0|1,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
NA21141,3|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
NA21142,0|0,0|0,0|0,0|0,0|0,0|0,0|1,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
NA21143,0|0,0|0,0|0,0|0,0|0,0|0,1|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [ ]:
ped_file = '[path]/integrated_call_samples.20130502.ALL.ped'
pedigree = pd.read_csv(ped_file, sep='\t', index_col='Individual ID')

In [ ]:
Y_train = pedigree.loc[genotypes.index]['Population']
Y_train.shape

(2504,)

In [ ]:
X = genotypes[:]
X.shape

(2504, 848)

In [ ]:
def get_max_genotype(g):
  v1, v2 = g.split("|")
  return max(int(v1), int(v2)) + 1

def key_gen(v1, v2):
  return f"{v1}|{v2}"

In [ ]:
genotype_keys = np.unique(X.values)
max_gen = max(map(get_max_genotype, genotype_keys))
onehot_encoding_depth = max_gen**2 + 1
MISSING_VALUE = [0 for _ in range(onehot_encoding_depth)]
MISSING_VALUE[-1] = 1
genotype_keys = [key_gen(i,j) for i in range(max_gen) for j in range(max_gen)]
replacement_dict = {k:i for i,k in enumerate(genotype_keys)}
reverse_replacement_dict = {v:k for k,v in replacement_dict.items()}
# replacement_dict

In [ ]:
X = X.replace(replacement_dict)
X.shape

(2504, 848)

## LD

In [ ]:
callset = allel.read_vcf(root_dir + 'ALL.chr22.mergedSV.v8.20130502.svs.genotypes.vcf')
g = allel.GenotypeArray(callset['calldata/GT'][callset['variants/CHROM'] == '22'])
gn = g.to_n_alt(fill=-1)
r = allel.rogers_huff_r(gn)
LD = squareform(r ** 2)
LD.shape

(848, 848)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(LD)

(array([553.,  97.,  81.,  19.,  98.]),
 array([0. , 0.2, 0.4, 0.6, 0.8, 1. ]),
 <a list of 5 Patch objects>)

In [ ]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
plt.hist(np.amax(LD, axis=1), bins=bins)

In [ ]:
LD_max_freqs = np.amax(LD, axis=1)
bin_labels = np.digitize(LD_max_freqs, bins=bins, right=True)
bin_general_labels, bin_counts = np.unique(bin_labels, return_counts=True)
bin_general_labels, bin_counts

(array([1, 2, 3, 4, 5]), array([553,  97,  81,  19,  98]))

In [ ]:
# hyperparameters
feature_size = X.shape[1]
inChannel = onehot_encoding_depth
# optimizer learning rate
learning_rate = 0.001
epochs = 60 
#epochs = 100   # chr20 LOS 5K


# training batch size
#batch_size = 32   # u19, 4984 samples
bs = 32


lr = 1e-3



# l1 regulalization
kr = 1e-4
k_initial = 'glorot_uniform'


# channel = inChannel
channel = inChannel

ndf_num = 128
kernel_len = 32
num_latent = ndf_num*4
p_size = 2



#dr_rate = drop_prec
dr_rate = 0.2  # avoid overfitting for missing ratio of 0.7

## Model

In [ ]:
# AE model in one cell
# # V1: verify our own ae model with yeast genotype data

# # 2.2.8 build variatial autoencoder for snp with subclassing function. 

class SNP_ENCODER(tf.keras.Model):    
    def __init__(self, feature_size, channel=channel, ndf=ndf_num, kernel_len=kernel_len, n_latent=num_latent, dr=dr_rate):    
        super(SNP_ENCODER, self).__init__()
        self.feature_size = feature_size
        self.channel = channel
        self.ndf = ndf
        self.n_latent = n_latent
        self.dr = dr # dropout rate

        # object, can be saved in tf mode
        self.stride=1
        self.kl = kernel_len

        
    def build(self, inputs): 
        #encoder
        # dense layer 1
        self.c1 = layers.Conv1D(filters=self.ndf, kernel_size=self.kl, strides=self.stride, padding="same", 
                      activation='relu', use_bias=True, 
                      kernel_initializer=k_initial, kernel_regularizer=tf.keras.regularizers.L1(kr), 
                      input_shape=(self.feature_size, self.channel))
        self.p1 = layers.MaxPooling1D(pool_size=p_size)
        self.drop1 = layers.Dropout(rate=self.dr)
        
        # dense layer 2
        self.c2 = layers.Conv1D(filters=(2*self.ndf), kernel_size=self.kl, strides=self.stride, padding="same", 
                      activation='relu', use_bias=True, kernel_initializer=k_initial, 
                      kernel_regularizer=tf.keras.regularizers.L1(kr))
        self.p2 = layers.MaxPooling1D(pool_size=p_size)
        self.drop2 = layers.Dropout(rate=self.dr)
        
        # dense layer 3
        self.c3 = layers.Conv1D(filters=(4*self.ndf), kernel_size=self.kl, strides=1, padding="same", 
                      activation='relu', use_bias=True, kernel_initializer=k_initial, 
                      kernel_regularizer=tf.keras.regularizers.L1(kr))
        self.p3 = layers.MaxPooling1D(pool_size=p_size)
        self.drop3 = layers.Dropout(rate=self.dr)

        super(SNP_ENCODER, self).build(inputs)
    
    def call(self, inputs, training=True):
        #print('SNP_ENCODER training flag: ', training)
        x = self.c1(inputs)
        x = self.p1(x)
        x = self.drop1(x, training=training)

        x = self.c2(x)
        x = self.p2(x)
        x = self.drop2(x, training=training)
        
        x = self.c3(x)
        return x

    # AFAIK: The most convenient method to print model.summary() 
    # similar to the sequential or functional API like.
    def build_graph(self):
        x = layers.Input(shape=(self.feature_size, self.channel))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))
    
    
    
# SNP_DECODER(keras.Model):   
class SNP_DECODER(tf.keras.Model):     
    def __init__(self, feature_size, channel=channel, ndf=ndf_num, kernel_len=kernel_len, n_latent=num_latent, dr=dr_rate):
        super(SNP_DECODER, self).__init__()
        self.feature_size = feature_size
        self.channel = channel
        self.ndf = ndf
        self.n_latent = n_latent
        self.dr = dr # dropout rate
        
        # object, can be saved in tf mode
        self.stride=1
        self.kl=kernel_len
        
    def build(self, inputs):
        #decoder        
        self.c1 = layers.Conv1D(filters=(2*self.ndf), kernel_size=self.kl, strides=self.stride, padding="same", 
                      activation='relu', use_bias=True, 
                      kernel_initializer=k_initial, kernel_regularizer=tf.keras.regularizers.L1(kr),
                      input_shape=((self.feature_size>>2), self.n_latent))
        
        
        self.s1 = layers.UpSampling1D(size=p_size)
        self.drop1 = layers.Dropout(rate=self.dr)
        
        # dense layer 2
        self.c2 = layers.Conv1D(filters=(1*self.ndf), kernel_size=self.kl, strides=self.stride, padding="same", 
                      activation='relu', use_bias=True, 
                      kernel_initializer=k_initial, kernel_regularizer=tf.keras.regularizers.L1(kr))

        self.s2 = layers.UpSampling1D(size=p_size)
        self.drop2 = layers.Dropout(rate=self.dr)
        

        # dense layer6
        self.c3 = layers.Conv1D(filters=self.channel, kernel_size=self.kl, strides=1, padding="same", 
                      activation='softmax', use_bias=True)
        
        super(SNP_DECODER, self).build(inputs)
        
    def call(self, inputs, training = True):
        #print('SNP_DECODER training flag: ', training)
        x = self.c1(inputs)
        x = self.s1(x)
        x = self.drop1(x, training=training)
        
        x = self.c2(x)
        x = self.s2(x)
        x = self.drop2(x, training=training)

        
        d_out = self.c3(x)
        return d_out
    
    # AFAIK: The most convenient method to print model.summary() 
    # similar to the sequential or functional API like.
    def build_graph(self):
        x = layers.Input(shape=(self.feature_size>>2, self.n_latent))
        
        return tf.keras.Model(inputs=[x], outputs=self.call(x))
    

    
#class SNP_AE(keras.Model):
class SNP_AE(tf.keras.Model):    
    def __init__(self, feature_size, channel=channel, ndf=ndf_num, n_latent=num_latent, dr=dr_rate):    
        super(SNP_AE, self).__init__()
        self.feature_size = feature_size
        self.channel = channel
        self.ndf = ndf
        self.n_latent = n_latent
        self.dr = dr # dropout rate      
        
        self.encoder = SNP_ENCODER(self.feature_size)
        self.decoder = SNP_DECODER(self.feature_size)
    

    def call(self, x, training=True): 
        latent = self.encoder(x, training)     
        res = self.decoder(latent, training)

        return res, latent
    
    # AFAIK: The most convenient method to print model.summary() 
    # similar to the sequential or functional API like.
    def build_graph(self):
        x = layers.Input(shape=(self.feature_size, self.channel))
        return tf.keras.Model(inputs=[x], outputs=self.call(x))
    

In [ ]:
# plot snp vae encoder model

SNP_encoder = SNP_ENCODER(feature_size)
SNP_encoder.build((None, feature_size, channel))
SNP_encoder.build_graph().summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 848, 37)]         0         
                                                                 
 conv1d_6 (Conv1D)           (None, 848, 128)          151680    
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 424, 128)         0         
 1D)                                                             
                                                                 
 dropout_5 (Dropout)         (None, 424, 128)          0         
                                                                 
 conv1d_7 (Conv1D)           (None, 424, 256)          1048832   
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 212, 256)         0         
 1D)                                                         

In [ ]:
# plot snp vae decoder model

SNP_decoder = SNP_DECODER(feature_size)

SNP_decoder.build((None, feature_size>>2, num_latent))
SNP_decoder.build_graph().summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 212, 512)]        0         
                                                                 
 conv1d_9 (Conv1D)           (None, 212, 256)          4194560   
                                                                 
 up_sampling1d_2 (UpSampling  (None, 424, 256)         0         
 1D)                                                             
                                                                 
 dropout_8 (Dropout)         (None, 424, 256)          0         
                                                                 
 conv1d_10 (Conv1D)          (None, 424, 128)          1048704   
                                                                 
 up_sampling1d_3 (UpSampling  (None, 848, 128)         0         
 1D)                                                       

In [ ]:
# plot snp vae model

SNP_ae = SNP_AE(feature_size)
SNP_ae.build((None, feature_size, channel))
SNP_ae.build_graph().summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 848, 37)]         0         
                                                                 
 snp_encoder_2 (SNP_ENCODER)  (None, 212, 512)         5395328   
                                                                 
 snp_decoder_2 (SNP_DECODER)  (None, 848, 37)          5394853   
                                                                 
Total params: 10,790,181
Trainable params: 10,790,181
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# AE model sub-functions

def generate_fake_missing(x_in, missing_ratio=0.5):

        # Generates missing genotypes
        # different missing loci for each individuals
        x_fake = x_in.copy()   # with .copy() to not overwrite the original data
        
        for i in range(x_in.shape[0]):
            missing_size = int(missing_ratio * x_in.shape[1])
          
            # without repeat random numbers: set replace with false
            missing_index = np.random.choice(x_in.shape[1], size=missing_size, replace=False)
            
            # missing loci are encoded as [0, 0]
            # x_fake[i, missing_index, :] = [0, 0, 1]  # yeast
            x_fake[i, missing_index, :] = MISSING_VALUE  # human

        return x_fake
        #return x_fake, x_in
    
    

def loss_function_cce(recon_x, x):
    # orders: y_true, y_pred
    cce = tf.keras.losses.categorical_crossentropy(x, recon_x)

    #cce = np.double(cce)
    cce = K.cast(cce, dtype='float32')

    lamb1 = 1.0
    loss = lamb1*cce
    #print('loss:', loss)
    
    loss = tf.reduce_mean(loss)
    #print('ave loss:', loss)
    
    return loss


## Training

In [ ]:
# With constraint
N_SPLITS=3
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = tf.keras.utils.to_categorical(X.to_numpy(), inChannel)
print(_x.shape)
_y = Y_train.to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  accuracies = []
  print(f"Training using fold {fold}")
  print("*******************************************")
  print("*******************************************")
  
  x_train, y_train, test_dataset, test_indices = _x[train_index], _y[train_index], (_x[test_index], _y[test_index]),Y_train.index[test_index]
  x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,
                                      stratify=y_train)
  
  
  for missing_ratio in [
                        0.05,
                        0.1,
                        0.2
                        ]:
    train_X = np.copy(x_train)
    valid_X = np.copy(x_valid)
    print(f"Missing rate {missing_ratio}")
    print("=====================================================")
    train_X_fake = generate_fake_missing(train_X, missing_ratio)

    diff = np.absolute(np.array(train_X) - np.array(train_X_fake))
    print('train_X_fake diff:', np.sum(diff))


    valid_X_fake = generate_fake_missing(valid_X, missing_ratio)
    diff = np.absolute(np.array(valid_X) - np.array(valid_X_fake))
    print('valid_X_fake diff:', np.sum(diff))


    K.clear_session()
    with strategy.scope():
      model = SNP_AE(feature_size)
      optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

      train_loss_metric = tf.keras.metrics.CategoricalCrossentropy()
      val_loss_metric = tf.keras.metrics.CategoricalCrossentropy()

      train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
      val_acc_metric = tf.keras.metrics.CategoricalAccuracy()

      for epoch in range(epochs):
        print('Start of epoch %d' % (epoch,))

        #----shuffle train data and lablel for each epoch-----------------------------------------------------------------------------------#    
        # shuffle data and labels at the same time
        idx = train_X.shape[0]


        indices = tf.range(start=0, limit=idx, dtype=tf.int32)

        shuffled_indices = tf.random.shuffle(indices)  
      
        train_X_fake = tf.gather(train_X_fake, shuffled_indices)
        train_X = tf.gather(train_X, shuffled_indices)        
        
        snp_x = tf.data.Dataset.from_tensor_slices(train_X_fake).batch(bs, drop_remainder=True) 
        snp_y = tf.data.Dataset.from_tensor_slices(train_X).batch(bs, drop_remainder=True)

        snp_x_v = tf.data.Dataset.from_tensor_slices(valid_X_fake).batch(bs, drop_remainder=True)
        snp_y_v = tf.data.Dataset.from_tensor_slices(valid_X).batch(bs, drop_remainder=True)

        loss_batch = []

        # Iterate over the batches of the dataset.
        for step, (snp_fake_batch, snp_label_batch) in enumerate(zip(snp_x, snp_y)): 
          with tf.GradientTape() as tape:
              recon_inputs, latents= model(snp_fake_batch, training=True)
              loss = loss_function_cce(recon_inputs, snp_label_batch)


          grads = tape.gradient(loss, model.trainable_variables)
          optimizer.apply_gradients(zip(grads, model.trainable_variables))    

          loss_batch.append(loss.numpy())


          # Update training metric.
          train_acc_metric.update_state(snp_label_batch, recon_inputs)
          train_loss_metric.update_state(snp_label_batch, recon_inputs)


        # Display metrics at the end of each epoch.
        train_loss = train_loss_metric.result()
        print("Training loss over epoch: ", epoch, train_loss.numpy())

        train_acc = train_acc_metric.result()
        print("Training acc over epoch: ", epoch, train_acc.numpy())


        # Reset training metrics at the end of each epoch
        train_loss_metric.reset_states()
        train_acc_metric.reset_states()


        # Run a validation loop at the end of each epoch.
        for x_batch_val, y_batch_val in zip(snp_x_v, snp_y_v):   

            val_recons, latents = model(x_batch_val, training=False)
            # Update val metrics
            val_loss_metric.update_state(y_batch_val, val_recons)
            val_acc_metric.update_state(y_batch_val, val_recons)

        val_loss = val_loss_metric.result()
        val_acc = val_acc_metric.result()

        val_loss_metric.reset_states()
        val_acc_metric.reset_states()
        #print("Validation acc: %.4f" % (float(val_acc),))
        print("Validation loss: ", epoch, val_loss.numpy())
        print("Validation acc: ", epoch, val_acc.numpy())


        #print('epoch %s: batch loss = %s' % (epoch, loss_batch))
        loss_epoch = np.mean(loss_batch)    
        print('epoch %s: loss = %s' % (epoch, loss_epoch))    
    
    save_name = f"[path]/Chr.22.ALL/AE/preds_mixed_mr_{missing_ratio}_fold_{fold}_.csv"
    avg_accuracy = []
    preds = []
    true_labels = []
    
    to_save_array = np.zeros((test_dataset[0].shape[0], test_dataset[0].shape[1]), dtype=object)
    test_X_missing = np.copy(test_dataset[0])
    for i in tqdm(list(range(test_dataset[0].shape[0]))):
      missing_index, _ = train_test_split(np.arange(x_train.shape[1]), train_size=missing_ratio,
                                    random_state=i + fold,
                                    shuffle=True,
                                    stratify=bin_labels
                                    )
      test_X_missing[i:i+1, missing_index, :] = MISSING_VALUE
      # predict
      predict_onehot, _ = model(test_X_missing[i:i+1, :, :], training=False)
      predict_onehot = predict_onehot.numpy()
      # only care the missing position
      predict_missing_onehot = predict_onehot[0:1, missing_index, :]
      # predict label
      predict_missing = np.argmax(predict_missing_onehot, axis=2)
      
      preds.extend(predict_missing.ravel().tolist())
      
      predict_haplotypes = np.argmax(predict_onehot, axis=2)
      # Only for haploids
      to_save_array[i] = predict_haplotypes
      # real label
      label_missing_onehot = np.argmax(test_dataset[0][i:i + 1, missing_index], axis=2)
      label_missing = np.argmax(test_dataset[0][i:i + 1, missing_index], axis=2)
      true_labels.extend(label_missing.ravel().tolist())
      # accuracy
      correct_prediction = np.equal(predict_missing, label_missing)
      accuracy = np.mean(correct_prediction)

      avg_accuracy.append(accuracy)

    df = pd.DataFrame(to_save_array, columns= headers[:], index = Y_train.index[test_index])
    df.to_csv(save_name)
    print('The average imputation accuracy' \
          'on test data with {} missing genotypes is {:.4f}: '
        .format(missing_ratio, np.mean(avg_accuracy)))
    cnf_matrix = confusion_matrix(true_labels, preds)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    print(f"Sensitivity: {np.mean(TPR)}")
    print(f"Specificity: {np.mean(TNR)}")
    print(f"F1-score macro: {f1_score(true_labels, preds, average='macro')}")
    print(f"F1-score micro: {f1_score(true_labels, preds, average='micro')}")
    accuracies.append(np.mean(avg_accuracy))
        
    

(2504, 848, 37)
Training using fold 1
*******************************************
*******************************************
Missing rate 0.05
train_X_fake diff: 126168.0
valid_X_fake diff: 14028.0
Start of epoch 0
Training loss over epoch:  0 0.99226964
Training acc over epoch:  0 0.8216968
Validation loss:  0 0.28677747
Validation acc:  0 0.94966096
epoch 0: loss = 0.9922699
Start of epoch 1
Training loss over epoch:  1 0.28748706
Training acc over epoch:  1 0.94945747
Validation loss:  1 0.27584726
Validation acc:  1 0.94966096
epoch 1: loss = 0.28748712
Start of epoch 2
Training loss over epoch:  2 0.27628425
Training acc over epoch:  2 0.9494399
Validation loss:  2 0.2638351
Validation acc:  2 0.94966096
epoch 2: loss = 0.27628422
Start of epoch 3
Training loss over epoch:  3 0.26254973
Training acc over epoch:  3 0.94943506
Validation loss:  3 0.24729724
Validation acc:  3 0.94966096
epoch 3: loss = 0.26254973
Start of epoch 4
Training loss over epoch:  4 0.24572328
Training acc

100%|██████████| 835/835 [00:11<00:00, 74.97it/s]


The average imputation accuracyon test data with 0.05 missing genotypes is 0.9503: 
Sensitivity: nan
Specificity: 0.9247459322091413
F1-score macro: 0.1957550233047236
F1-score micro: 0.9502708867978329
Missing rate 0.1


<ipython-input-149-6db1d8498207>:190: RuntimeWarning: invalid value encountered in true_divide
  TPR = TP/(TP+FN)


train_X_fake diff: 252336.0
valid_X_fake diff: 28056.0
Start of epoch 0
Training loss over epoch:  0 1.0077953
Training acc over epoch:  0 0.840729
Validation loss:  0 0.28634626
Validation acc:  0 0.94966096
epoch 0: loss = 1.0077955
Start of epoch 1
Training loss over epoch:  1 0.27801722
Training acc over epoch:  1 0.9494302
Validation loss:  1 0.25423872
Validation acc:  1 0.94966096
epoch 1: loss = 0.27801713
Start of epoch 2
Training loss over epoch:  2 0.24932674
Training acc over epoch:  2 0.949415
Validation loss:  2 0.23421755
Validation acc:  2 0.94966096
epoch 2: loss = 0.24932675
Start of epoch 3
Training loss over epoch:  3 0.23599026
Training acc over epoch:  3 0.9494479
Validation loss:  3 0.22493635
Validation acc:  3 0.94966096
epoch 3: loss = 0.23599026
Start of epoch 4
Training loss over epoch:  4 0.22614619
Training acc over epoch:  4 0.9494591
Validation loss:  4 0.21120174
Validation acc:  4 0.94969046
epoch 4: loss = 0.22614622
Start of epoch 5
Training loss ove

100%|██████████| 835/835 [00:11<00:00, 75.10it/s]


The average imputation accuracyon test data with 0.1 missing genotypes is 0.9509: 
Sensitivity: 0.18064045407514415
Specificity: 0.9395821403491156
F1-score macro: 0.2093057608839328
F1-score micro: 0.9509409751924722
Missing rate 0.2
train_X_fake diff: 507676.0
valid_X_fake diff: 56446.0
Start of epoch 0
Training loss over epoch:  0 1.0259941
Training acc over epoch:  0 0.82111925
Validation loss:  0 0.2898954
Validation acc:  0 0.94966096
epoch 0: loss = 1.025994
Start of epoch 1
Training loss over epoch:  1 0.2800139
Training acc over epoch:  1 0.94946146
Validation loss:  1 0.26235276
Validation acc:  1 0.94966096
epoch 1: loss = 0.28001386
Start of epoch 2
Training loss over epoch:  2 0.25922915
Training acc over epoch:  2 0.94942707
Validation loss:  2 0.24491425
Validation acc:  2 0.94966096
epoch 2: loss = 0.25922915
Start of epoch 3
Training loss over epoch:  3 0.2469185
Training acc over epoch:  3 0.9494302
Validation loss:  3 0.23654099
Validation acc:  3 0.94966096
epoch 3:

100%|██████████| 835/835 [00:11<00:00, 75.14it/s]


The average imputation accuracyon test data with 0.2 missing genotypes is 0.9532: 
Sensitivity: 0.11678529248789078
Specificity: 0.957190665553605
F1-score macro: 0.13832989999134732
F1-score micro: 0.9531800304716012
Training using fold 2
*******************************************
*******************************************
Missing rate 0.05
train_X_fake diff: 126168.0
valid_X_fake diff: 14028.0
Start of epoch 0
Training loss over epoch:  0 0.87990344
Training acc over epoch:  0 0.88961077
Validation loss:  0 0.2917544
Validation acc:  0 0.94892395
epoch 0: loss = 0.8799034
Start of epoch 1
Training loss over epoch:  1 0.28441942
Training acc over epoch:  1 0.94921315
Validation loss:  1 0.27215222
Validation acc:  1 0.94892395
epoch 1: loss = 0.28441945
Start of epoch 2
Training loss over epoch:  2 0.2622411
Training acc over epoch:  2 0.94927645
Validation loss:  2 0.2522594
Validation acc:  2 0.94892395
epoch 2: loss = 0.2622411
Start of epoch 3
Training loss over epoch:  3 0.2508

100%|██████████| 835/835 [00:11<00:00, 75.77it/s]


The average imputation accuracyon test data with 0.05 missing genotypes is 0.9531: 
Sensitivity: 0.274700892643173
Specificity: 0.9314055208883303
F1-score macro: 0.32350968151227877
F1-score micro: 0.9530652979754776
Missing rate 0.1
train_X_fake diff: 252336.0
valid_X_fake diff: 28056.0
Start of epoch 0
Training loss over epoch:  0 1.7409554
Training acc over epoch:  0 0.68188816
Validation loss:  0 0.29982334
Validation acc:  0 0.94892395
epoch 0: loss = 1.7409554
Start of epoch 1
Training loss over epoch:  1 0.28195128
Training acc over epoch:  1 0.9492644
Validation loss:  1 0.27502963
Validation acc:  1 0.94892395
epoch 1: loss = 0.28195134
Start of epoch 2
Training loss over epoch:  2 0.27153024
Training acc over epoch:  2 0.9492716
Validation loss:  2 0.2659143
Validation acc:  2 0.94892395
epoch 2: loss = 0.27153027
Start of epoch 3
Training loss over epoch:  3 0.26489744
Training acc over epoch:  3 0.9493541
Validation loss:  3 0.26284808
Validation acc:  3 0.94892395
epoch 3

100%|██████████| 835/835 [00:10<00:00, 76.12it/s]


The average imputation accuracyon test data with 0.1 missing genotypes is 0.9503: 
Sensitivity: 0.07222773796628683
Specificity: 0.9348977841771137
F1-score macro: 0.07464877976796348
F1-score micro: 0.9502994011976048
Missing rate 0.2
train_X_fake diff: 507676.0
valid_X_fake diff: 56446.0
Start of epoch 0
Training loss over epoch:  0 1.5265919
Training acc over epoch:  0 0.6872629
Validation loss:  0 0.29326478
Validation acc:  0 0.94892395
epoch 0: loss = 1.5265918
Start of epoch 1
Training loss over epoch:  1 0.2908153
Training acc over epoch:  1 0.94927806
Validation loss:  1 0.273048
Validation acc:  1 0.94892395
epoch 1: loss = 0.29081523
Start of epoch 2
Training loss over epoch:  2 0.26943335
Training acc over epoch:  2 0.9492684
Validation loss:  2 0.25719884
Validation acc:  2 0.94892395
epoch 2: loss = 0.26943338
Start of epoch 3
Training loss over epoch:  3 0.25678423
Training acc over epoch:  3 0.9492732
Validation loss:  3 0.24995953
Validation acc:  3 0.94892395
epoch 3:

100%|██████████| 835/835 [00:11<00:00, 75.51it/s]


The average imputation accuracyon test data with 0.2 missing genotypes is 0.9531: 
Sensitivity: 0.1264710204301507
Specificity: 0.9558640240545643
F1-score macro: 0.15392476890481385
F1-score micro: 0.9531091662828189
Training using fold 3
*******************************************
*******************************************
Missing rate 0.05
train_X_fake diff: 126252.0
valid_X_fake diff: 14028.0
Start of epoch 0
Training loss over epoch:  0 1.2901678
Training acc over epoch:  0 0.75845337
Validation loss:  0 0.29297647
Validation acc:  0 0.9494767
epoch 0: loss = 1.2901678
Start of epoch 1
Training loss over epoch:  1 0.28446037
Training acc over epoch:  1 0.9492436
Validation loss:  1 0.26839107
Validation acc:  1 0.9494767
epoch 1: loss = 0.2844604
Start of epoch 2
Training loss over epoch:  2 0.2654121
Training acc over epoch:  2 0.94920594
Validation loss:  2 0.25190634
Validation acc:  2 0.9494767
epoch 2: loss = 0.2654121
Start of epoch 3
Training loss over epoch:  3 0.25273263

100%|██████████| 834/834 [00:10<00:00, 76.72it/s]


The average imputation accuracyon test data with 0.05 missing genotypes is 0.9511: 
Sensitivity: 0.15709974932064313
Specificity: 0.938108121040098
F1-score macro: 0.18848827601971244
F1-score micro: 0.9510962658444673
Missing rate 0.1
train_X_fake diff: 252504.0
valid_X_fake diff: 28056.0
Start of epoch 0
Training loss over epoch:  0 1.4849354
Training acc over epoch:  0 0.7199613
Validation loss:  0 0.29120836
Validation acc:  0 0.9494767
epoch 0: loss = 1.484935
Start of epoch 1
Training loss over epoch:  1 0.28113443
Training acc over epoch:  1 0.94925
Validation loss:  1 0.26292506
Validation acc:  1 0.9494767
epoch 1: loss = 0.28113437
Start of epoch 2
Training loss over epoch:  2 0.26064447
Training acc over epoch:  2 0.9491835
Validation loss:  2 0.24882446
Validation acc:  2 0.9494767
epoch 2: loss = 0.26064447
Start of epoch 3
Training loss over epoch:  3 0.25256747
Training acc over epoch:  3 0.94923556
Validation loss:  3 0.24428228
Validation acc:  3 0.9494767
epoch 3: los

100%|██████████| 834/834 [00:10<00:00, 76.00it/s]


The average imputation accuracyon test data with 0.1 missing genotypes is 0.9526: 
Sensitivity: 0.14721787088096427
Specificity: 0.9440126864391516
F1-score macro: 0.1784695841415567
F1-score micro: 0.9525665182140002
Missing rate 0.2
train_X_fake diff: 508014.0
valid_X_fake diff: 56446.0
Start of epoch 0
Training loss over epoch:  0 0.82743955
Training acc over epoch:  0 0.89998925
Validation loss:  0 0.29126823
Validation acc:  0 0.9494767
epoch 0: loss = 0.8274395
Start of epoch 1
Training loss over epoch:  1 0.2867021
Training acc over epoch:  1 0.9491915
Validation loss:  1 0.27279198
Validation acc:  1 0.9494767
epoch 1: loss = 0.2867021
Start of epoch 2
Training loss over epoch:  2 0.2661549
Training acc over epoch:  2 0.9491843
Validation loss:  2 0.2506277
Validation acc:  2 0.9494767
epoch 2: loss = 0.26615483
Start of epoch 3
Training loss over epoch:  3 0.25050694
Training acc over epoch:  3 0.9492292
Validation loss:  3 0.24258618
Validation acc:  3 0.9494767
epoch 3: loss

100%|██████████| 834/834 [00:10<00:00, 76.21it/s]


The average imputation accuracyon test data with 0.2 missing genotypes is 0.9527: 
Sensitivity: nan
Specificity: 0.9576856985968495


<ipython-input-149-6db1d8498207>:190: RuntimeWarning: invalid value encountered in true_divide
  TPR = TP/(TP+FN)


F1-score macro: 0.15162445079339731
F1-score micro: 0.9526769117250579


In [ ]:
to_save_array.shape, Y_train.index[test_index].shape, headers[:].shape

((834, 848), (834,), (848,))